In [30]:
import numpy as np
import pandas as pd

In [89]:
cd /content/drive/My Drive/ml/wion_crawling

/content/drive/My Drive/ml/wion_crawling


In [90]:
BS = pd.read_csv('BS.csv')
CF = pd.read_csv('CF.csv')
IS = pd.read_csv('IS.csv')

def find_index(index, df, df_1):
  x = list(df.iloc[:, 1])
  for i in range(len(x)):
    if x[i] == index:
      break
  p = df.iloc[i, 1:]

  temp = pd.DataFrame(p).T
  temp.columns = ['단위:억 원', '2015', '2016', '2017', '2018', '2019']
  
  df_1 = pd.concat([df_1, temp], axis = 0, ignore_index=True)
  return df_1

* 직접 받아와야 하는 항목: 매출액, 매출원가, 판관비, 감가상각비, 기타무형자산상각비, 이자수익(금융수익 -> 있는걸로), 현금및현금성자산, 단기금융자산, 이자비용(금융원가), 기타영업외수익, 기타영업외 비용, 관계사, tax

매출총이익, 영업이익,
* 계산이 가능한 항목: 감가상각비(+기타무형자산상각비), EBITDA, 영업외손익, 합계, 이자부채, pretax income, 당기순이익



## IS

In [91]:
essential_index_IS = ['매출액(수익)', '매출원가', '매출총이익',
                    '판매비와관리비', '영업이익', '기타무형자산상각비',
                    '감가상각비','금융수익', '금융원가', '기타영업외수익', '기타영업외비용', 
                    '종속기업,공동지배기업및관계기업관련손익', '법인세비용차감전계속사업이익','법인세비용', '당기순이익']
result_is = pd.DataFrame(columns=['단위:억 원', '2015', '2016', '2017', '2018', '2019'])

for i in essential_index_IS:
  result_is = find_index(i, IS, result_is)

# nan to 0
result_is = result_is.fillna(0)

# 감가상각 합치기
result_is.iloc[6, 1:] = result_is.iloc[6, 1:] + result_is.iloc[5, 1:]
result_is = result_is.drop(5)

In [92]:
# 금융수익 중복
X = list(IS.iloc[100, 2:])
for i in range(5):
  result_is.iloc[6, 1+i] = X[i]

## BS

In [93]:
essential_index_BS = ['현금및현금성자산', '단기금융자산']

result_bs = pd.DataFrame(columns=['단위:억 원', '2015', '2016', '2017', '2018', '2019'])
for i in essential_index_BS:
  result_bs = find_index(i, BS, result_bs)

In [94]:
X[1]

143.485671

In [95]:
result_is_bs = pd.concat([result_is, result_bs], ignore_index=True)
result_is_bs

,단위:억 원,2015,2016,2017,2018,2019
0,매출액(수익),912.78,2946.22,4646.29,5358.06,7015.92
1,매출원가,1147.85,2680.88,3328.29,3906.46,5095.84
2,매출총이익,-235.066,265.337,1318,1451.6,1920.07
3,판매비와관리비,1801.36,569.563,658.207,894.812,1002.65
4,영업이익,-2036.42,-304.226,659.794,556.787,917.424
5,감가상각비,156.135,60.759,94.9511,146.154,92.5178
6,금융수익,68.4472,143.486,232.198,193.208,275.407
7,금융원가,18412,867.831,763.453,243.372,386.826
8,기타영업외수익,26.6358,12.1783,8.89983,3903.99,40.0349
9,기타영업외비용,8.21253,0.390102,1.36243,87.4827,21.0878


## Result

In [96]:
result = pd.DataFrame(columns=['단위:억 원', '2015', '2016', '2017', '2018', '2019'])
financial_index = ['매출액(수익)', 'yoy', '매출원가', 'COGS', '매출총이익', 'GPM',
                   '판매비와관리비', 'SG&A', '영업이익', 'OPM',
                   '감가상각비', 'Dep', 'EBITDA', '마진율',
                   '금융수익', '현금및현금성자산',
                   '단기금융자산', '합계', '매출액대비', '금융원가', 
                   '기타영업외수익', '매출액대비', '기타영업외비용', 
                   '매출액대비', '종속기업,공동지배기업및관계기업관련손익', '매출액대비', '법인세비용차감전계속사업이익', '매출액대비',
                   '법인세비용', '세전이익대비', '당기순이익', 'NPM']
essential_index = ['매출액(수익)', '매출원가', '매출총이익',
                    '판매비와관리비', '영업이익', '현금및현금성자산', '단기금융자산'
                    '감가상각비','금융수익', '금융원가', '기타영업외수익', '기타영업외비용', 
                    '종속기업,공동지배기업및관계기업관련손익', '법인세비용차감전계속사업이익','법인세비용', '당기순이익']

In [97]:
def yoy(index_value_list):
  R = ['yoy', 0]

  for i in range(2, len(index_value_list)):
    try:
      R.append(
          (index_value_list[i] - index_value_list[i-1]) / index_value_list[i-1] * 100
          )
    except:
      R.append(0)
  temp = pd.DataFrame(R).T
  temp.columns = ['단위:억 원', '2015', '2016', '2017', '2018', '2019']

  return temp

def append_financial_data(is_bs, target, result):
  for i in range(len(is_bs)):
    if is_bs.iloc[i, 0] == target:
      temp = pd.DataFrame(list(is_bs.iloc[i])).T
      temp.columns = ['단위:억 원', '2015', '2016', '2017', '2018', '2019']
      result = pd.concat([result, temp], ignore_index=True)

  return result
  
def per_take(take, before_list, target, result):
  R = [target]
  for i in range(1, len(take)):
    R.append(before_list[i] / take[i] * 100)
    
  temp = pd.DataFrame(R).T
  temp.columns = ['단위:억 원', '2015', '2016', '2017', '2018', '2019']

  result = pd.concat([result, temp], ignore_index=True)

  return result

In [98]:
for i in range(len(financial_index)):
  if financial_index[i] in essential_index:
    result = append_financial_data(result_is_bs, financial_index[i], result)
  elif financial_index[i] == 'yoy':
    temp = yoy(list(result.iloc[-1]))
    result = pd.concat([result, temp], ignore_index=True)
  else:
    temp = pd.DataFrame([financial_index[i], 0, 0, 0, 0, 0]).T
    temp.columns = ['단위:억 원', '2015', '2016', '2017', '2018', '2019']
    result = pd.concat([result, temp], 
                       ignore_index= True)

result.iloc[10, 1:] = result_is_bs.iloc[5, 1:] # 감가상각비
result.iloc[16, 1:] = result_is_bs.iloc[15, 1:] # 단기금융자산
result = result.fillna(0)
for i in range(1, 6):
  result.iloc[17, i] = result.iloc[15, i] + result.iloc[16, i] # 합계
  result.iloc[12, i] = result.iloc[8, i] + result.iloc[10, i] # ebitda

# 세전 이익 대비
for i in range(1, 6):
  result.iloc[29, i] = result.iloc[28, i] / result.iloc[26, i] * 100

## ====== 아래는 매출액 대비
per_take_list = ['COGS', 'GPM', 'SG&A', 'OPM', 'Dep', '마진율', '매출액대비', 'NPM']

take = result.iloc[0]
for i in range(len(result)):
  if result.iloc[i, 0] in per_take_list:
    for j in range(1, 6):
      result.iloc[i, j] = result.iloc[i-1, j] / take[j] * 100

In [99]:
result.to_csv('/content/drive/My Drive/ml/wion_crawling/modeling.csv')

In [100]:
result

,단위:억 원,2015,2016,2017,2018,2019
0,매출액(수익),912.780290,2946.220208,4646.293341,5358.058011,7015.918598
1,yoy,0.000000,222.774302,57.703532,15.318978,30.941445
2,매출원가,1147.846170,2680.883576,3328.292732,3906.458594,5095.844251
3,COGS,125.752734,90.993999,71.633289,72.908106,72.632602
4,매출총이익,-235.065880,265.336631,1318.000610,1451.599417,1920.074346
5,GPM,-25.752734,9.006001,28.366711,27.091894,27.367398
6,판매비와관리비,1801.358560,569.563017,658.206585,894.811971,1002.650474
7,SG&A,197.348538,19.331991,14.166273,16.700304,14.291079
8,영업이익,-2036.424440,-304.226386,659.794024,556.787445,917.423872
9,OPM,-223.101272,-10.325989,14.200438,10.391590,13.076319
